## Choosing a pre training model

In [4]:

%tensorflow_version 1.x # Select module of the tensorflow
# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
    }
}

selected_model = 'ssd_mobilenet_v2'


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x # Select module of the tensorflow`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


## Installing Required Packages 

In [5]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -qq Cython contextlib2 pillow lxml matplotlib

!pip install -qq pycocotools

## General imports

In [6]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv
import re
import cv2 
import os
import glob
import xml.etree.ElementTree as ET

import io
import tensorflow.compat.v1 as tf

from PIL import Image
from collections import namedtuple, OrderedDict

import shutil
import urllib.request
import tarfile

from google.colab import files

In [7]:
#tenorflow must be v1.15.2, because object detection API is removed from tf v 2.0+
print(tf.__version__)

1.15.2


## Downloading and Orgniazing Images and Annotations

In [8]:
!git clone https://github.com/Dovahkiin169/CareLabelsData.git

Cloning into 'CareLabelsData'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 572 (delta 118), reused 121 (delta 99), pack-reused 432
Receiving objects: 100% (572/572), 190.25 MiB | 53.55 MiB/s, done.
Resolving deltas: 100% (330/330), done.


In [9]:
cd CareLabelsData

/content/CareLabelsData


## Preprocessing Images and Labels
1. Converting annotations from xml's to two csv files for `train_labels/` and `train_labels/`.
2. Creating a label map file that specifies the number of class (one class in this case)

In [10]:
%cd /content/CareLabelsData/data
images_extension = 'jpg'

def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text + '.' + images_extension,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")

pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n    display_name: '{1}'\n }}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)


/content/CareLabelsData/data
Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


In [12]:
!cat label_map.pbtxt

item {
    id: 1
    name: 'bleaching_with_chlorine_allowed'
    display_name: 'bleaching_with_chlorine_allowed'
 }

 item {
    id: 2
    name: 'chlorine_and_non_chlorine_bleach'
    display_name: 'chlorine_and_non_chlorine_bleach'
 }

 item {
    id: 3
    name: 'do_not_bleach'
    display_name: 'do_not_bleach'
 }

 item {
    id: 4
    name: 'do_not_dry_clean'
    display_name: 'do_not_dry_clean'
 }

  item {
    id: 5
    name: 'do_not_iron'
    display_name: 'do_not_iron'
 }

  item {
    id: 6
    name: 'do_not_tumble_drying'
    display_name: 'do_not_tumble_drying'
 }

  item {
    id: 7
    name: 'do_not_wash'
    display_name: 'do_not_wash'
 }

  item {
    id: 8
    name: 'do_not_wet_clean'
    display_name: 'do_not_wet_clean'
 }

  item {
    id: 9
    name: 'drip_dry_in_the_shade'
    display_name: 'drip_dry_in_the_shade'
 }

  item {
    id: 10
    name: 'drip_dry'
    display_name: 'drip_dry'
 }

  item {
    id: 11
    name: 'dry_clean_any_solvent'
    display_name: 'dry

## Downloading and Preparing Tensorflow model
1. Cloning [Tensorflow models](https://github.com/tensorflow/models.git). This repo contains object detection API. 
2. Compiling the protos and adding folders to the os environment.
3. Testing the model builder.

In [13]:
%cd /content/CareLabelsData/
!git clone --q https://github.com/tensorflow/models.git

/content/CareLabelsData


In [14]:
%cd /content/CareLabelsData/models/research
!protoc object_detection/protos/*.proto --python_out=.

os.environ['PYTHONPATH'] += ':/content/CareLabelsData/models/research/:/content/CareLabelsData/models/research/slim/'

/content/CareLabelsData/models/research


In [15]:
!pip3 install tf_slim
!python3 object_detection/builders/model_builder_test.py

     |████████████████████████████████| 358kB 5.6MB/s 


## Generating Tf record
- Generating TFRecords files for training and testing csv's.
- Tensorflow accepts the data as tfrecords which is a binary file that run fast with low memory usage.

In [16]:
from object_detection.utils import dataset_util
%cd /content/CareLabelsData/models/

DATA_BASE_PATH = '/content/CareLabelsData/data/'
image_dir = '/content/CareLabelsData/data/images'

def class_text_to_int(row_label):
		if row_label == 'bleaching_with_chlorine_allowed':
				return 1
		elif row_label == 'chlorine_and_non_chlorine_bleach':
				return 2
		elif row_label == 'do_not_bleach':
				return 3
		elif row_label == 'do_not_dry_clean':
				return 4
		elif row_label == 'do_not_iron':
				return 5
		elif row_label == 'do_not_tumble_drying':
				return 6
		elif row_label == 'do_not_wash':
				return 7
		elif row_label == 'do_not_wet_clean':
				return 8
		elif row_label == 'drip_dry_in_the_shade':
				return 9
		elif row_label == 'drip_dry':
				return 10
		elif row_label == 'dry_clean_any_solvent':
				return 11
		elif row_label == 'dry_clean_hydrocarbon_solvent_only_hcs':
				return 12
		elif row_label == 'dry_clean_tetrachloroethylene_pce_only':
				return 13
		elif row_label == 'dry_flat_in_the_shade':
				return 14
		elif row_label == 'dry_flat':
				return 15
		elif row_label == 'dry_in_the_shade':
				return 16
		elif row_label == 'drying_symbol':
				return 17
		elif row_label == 'gentle_cleaning_with_hydrocarbon_solvents':
				return 18
		elif row_label == 'gentle_cleaning_with_pce':
				return 19
		elif row_label == 'gentle_wet_cleaning':
				return 20
		elif row_label == 'hand_wash':
				return 21
		elif row_label == 'ironing_at_high_temp':
				return 22
		elif row_label == 'ironing_at_low_temp':
				return 23
		elif row_label == 'ironing_at_med_temp':
				return 24
		elif row_label == 'ironing':
				return 25
		elif row_label == 'line_dry_in_the_shade':
				return 26
		elif row_label == 'line_dry':
				return 27
		elif row_label == 'no_steam':
				return 28
		elif row_label == 'non_chlorine_bleach_when_needed':
				return 29
		elif row_label == 'professional_cleaning':
				return 30
		elif row_label == 'professional_wet_cleaning':
				return 31
		elif row_label == 'tumble_drying_low_temps':
				return 32
		elif row_label == 'tumble_drying_normal':
				return 33
		elif row_label == 'tumble_drying':
				return 34
		elif row_label == 'very_gentle_cleaning_with_hydrocarbon_solvents':
				return 35
		elif row_label == 'very_gentle_cleaning_with_pce':
				return 36
		elif row_label == 'very_gentle_wet_cleaning':
				return 37
		elif row_label == 'wash_at_or_below_30_mild_fine_wash':
				return 38
		elif row_label == 'wash_at_or_below_30_very_mild_fine_wash':
				return 39
		elif row_label == 'wash_at_or_below_30':
				return 40
		elif row_label == 'wash_at_or_below_40_mild_fine_wash':
				return 41
		elif row_label == 'wash_at_or_below_40_very_mild_fine_wash':
				return 42
		elif row_label == 'wash_at_or_below_40':
				return 43
		elif row_label == 'wash_at_or_below_50_mild_fine_wash':
				return 44
		elif row_label == 'wash_at_or_below_50':
				return 45
		elif row_label == 'wash_at_or_below_60_mild_fine_wash':
				return 46
		elif row_label == 'wash_at_or_below_60':
				return 47
		elif row_label == 'wash_at_or_below_70':
				return 48
		elif row_label == 'wash_at_or_below_90':
				return 49
		elif row_label == 'washing_symbol':
				return 50			
		else:
				0

def split(df, group):
		data = namedtuple('data', ['filename', 'object'])
		gb = df.groupby(group)
		return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
		with tf.io.gfile.GFile(os.path.join(path, '{}'.format(os.path.splitext(group.filename)[0])), 'rb') as fid:
				encoded_jpg = fid.read()
		encoded_jpg_io = io.BytesIO(encoded_jpg)
		image = Image.open(encoded_jpg_io)
		width, height = image.size

		filename = group.filename.encode('utf8')
		xmins = []
		xmaxs = []
		ymins = []
		ymaxs = []
		classes_text = []
		classes = []

		for index, row in group.object.iterrows():
				xmins.append(row['xmin'] / width)
				xmaxs.append(row['xmax'] / width)
				ymins.append(row['ymin'] / height)
				ymaxs.append(row['ymax'] / height)
				classes_text.append(row['class'].encode('utf8'))
				classes.append(class_text_to_int(row['class']))

		tf_example = tf.train.Example(features=tf.train.Features(feature={
				'image/height': dataset_util.int64_feature(height),
				'image/width': dataset_util.int64_feature(width),
				'image/filename': dataset_util.bytes_feature(os.path.splitext(filename)[0]),
				'image/source_id': dataset_util.bytes_feature(os.path.splitext(filename)[0]),
				'image/encoded': dataset_util.bytes_feature(encoded_jpg),
				'image/format': dataset_util.bytes_feature(b'jpg'),
				'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
				'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
				'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
				'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
				'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
				'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
		return tf_example

for csv in ['train_labels', 'test_labels']:
  writer = tf.io.TFRecordWriter(DATA_BASE_PATH + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(DATA_BASE_PATH + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
      tf_example = create_tf_example(group, path)
      writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), DATA_BASE_PATH + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(DATA_BASE_PATH +csv + '.record'))


/content/CareLabelsData/models
Successfully created the TFRecords: /content/CareLabelsData/data/train_labels.record
Successfully created the TFRecords: /content/CareLabelsData/data/test_labels.record


In [17]:
!ls -lX /content/CareLabelsData/data/

total 66792
drwxr-xr-x 2 root root     4096 Sep 12 12:41 images
drwxr-xr-x 2 root root     4096 Sep 12 12:41 test_labels
drwxr-xr-x 2 root root     4096 Sep 12 12:41 train_labels
-rw-r--r-- 1 root root      929 Sep 12 12:42 test_labels.csv
-rw-r--r-- 1 root root    32735 Sep 12 12:42 train_labels.csv
-rw-r--r-- 1 root root     5092 Sep 12 12:42 label_map.pbtxt
-rw-r--r-- 1 root root  2006675 Sep 12 12:42 test_labels.record
-rw-r--r-- 1 root root 66329466 Sep 12 12:42 train_labels.record


## Downloading the Base Model
1. Downloading the selected model and extracting its content.
2. Creating a directory to save model while training.

In [18]:
%cd /content/CareLabelsData/models/research

MODEL = MODELS_CONFIG[selected_model]['model_name']

pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

MODEL_FILE = MODEL + '.tar.gz'

DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

fine_tune_dir = '/content/CareLabelsData/models/research/pretrained_model'

#checks if already downloaded
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(fine_tune_dir)):
    shutil.rmtree(fine_tune_dir)
os.rename(MODEL, fine_tune_dir)

/content/CareLabelsData/models/research


In [19]:
!echo {fine_tune_dir}
!ls -alh {fine_tune_dir}

/content/CareLabelsData/models/research/pretrained_model
total 135M
drwxr-xr-x  3 345018 89939 4.0K Mar 30  2018 .
drwxr-xr-x 24 root   root  4.0K Sep 12 12:43 ..
-rw-r--r--  1 345018 89939   77 Mar 30  2018 checkpoint
-rw-r--r--  1 345018 89939  67M Mar 30  2018 frozen_inference_graph.pb
-rw-r--r--  1 345018 89939  65M Mar 30  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 345018 89939  15K Mar 30  2018 model.ckpt.index
-rw-r--r--  1 345018 89939 3.4M Mar 30  2018 model.ckpt.meta
-rw-r--r--  1 345018 89939 4.2K Mar 30  2018 pipeline.config
drwxr-xr-x  3 345018 89939 4.0K Mar 30  2018 saved_model


## Configuring the Training Pipeline
1. Adding the path for the TFRecords files and pbtxt,batch_size,num_steps,num_classes to the configuration file.
2. Adding some Image augmentation.
3. Creating a directory to save the model at each checkpoint while training. 

In [20]:
CONFIG_BASE = "/content/CareLabelsData/models/research/object_detection/samples/configs/"

model_pipline = os.path.join(CONFIG_BASE, pipeline_file)
model_pipline

'/content/CareLabelsData/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config'

In [21]:
%%writefile {model_pipline}
model {
  ssd {
    num_classes: 50 # number of classes
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    # all images will be resized to height and width parametrs
    image_resizer { 
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: true # to counter over fitting
        dropout_keep_probability: 0.8
        kernel_size: 1
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
            weight: 0.001 # higher regularizition to counter overfitting
          }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {
            train: true,
            scale: true,
            center: true,
            decay: 0.9997,
            epsilon: 0.001,
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.001 # higher regularizition to counter overfitting
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000 
        iou_threshold: 0.95
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 3
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        
        #there are some images with more than one up to 3.
        #5 for future proof
        max_detections_per_class: 5
        #probably overkill, max 12 labels in one image 
        max_total_detections: 16
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  batch_size: 16 # training batch size
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.003
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }

  #the path to the pretrained model. 
  fine_tune_checkpoint: "/content/CareLabelsData/models/research/pretrained_model/model.ckpt"
  fine_tune_checkpoint_type:  "detection"
  num_steps: 10000 
  

  # data augmentaion will help the model generalize but training time will increase greatly  
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/content/CareLabelsData/data/train_labels.record"    #path to the training TFRecord
  }
  label_map_path: "/content/CareLabelsData/data/label_map.pbtxt"  #path to the label map 
}

eval_config: {
  num_examples: 3  # the number of images in your "testing" data
  num_visualizations: 3  # the number of images to disply in Tensorboard
  # Remove the below line to evaluate indefinitely.
  #max_evals: 10
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/CareLabelsData/data/test_labels.record"    #path to the testing TFRecord
  }
  label_map_path: "/content/CareLabelsData/data/label_map.pbtxt"  #path to the label map 
  shuffle: false
  num_readers: 1
}

Overwriting /content/CareLabelsData/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config


In [22]:
model_dir = 'training/'# dir where model will be saved at each checkpoint

# remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

## Tensorboard
1. Downlaoding and Unzipping
2. Creating a link

In [26]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-09-12 12:43:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.205.91.57, 52.201.131.65, 52.2.226.158, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.205.91.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  17.8MB/s    in 0.7s    

2020-09-12 12:43:19 (17.8 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [27]:
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

In [28]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://f5e584d64cdf.ngrok.io


## Training


In [ ]:
!python3 /content/CareLabelsData/models/research/object_detection/model_main.py \
    --pipeline_config_path={model_pipline}\
    --model_dir={model_dir} \
    --alsologtostderr \

W0912 12:43:29.566325 140357349037952 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: None
I0912 12:43:29.566520 140357349037952 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0912 12:43:29.566605 140357349037952 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0912 12:43:29.566694 140357349037952 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0912 12:43:29.566774 140357349037952 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0912 12:43:29.566874 140357349037952 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_t


## Exporting trained model



In [ ]:
output_directory = '/content/CareLabelsData/models/research/fine_tuned_model'# exported model location

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

!python /content/CareLabelsData/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={model_pipline} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [ ]:
files.download(output_directory + '/frozen_inference_graph.pb')

In [ ]:
files.download(DATA_BASE_PATH + '/label_map.pbtxt')